In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Imputer

In [2]:
testingSet = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/test_final_100k.csv')

In [3]:
testingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
id              100000 non-null int64
idaviso         100000 non-null int64
idpostulante    100000 non-null object
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


# Agrego informacion de avisos detalle al testingSet

In [4]:
avisos_detalle = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/optimized_avisos_detalle.csv')
avisos_detalle.drop('Unnamed: 0',inplace=True,axis=1)

In [5]:
avisos_detalle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25287 entries, 0 to 25286
Data columns (total 6 columns):
idaviso                      25287 non-null int64
nombre_area_code             25287 non-null int64
denominacion_empresa_code    25287 non-null int64
nivel_laboral_code           25287 non-null int64
tipo_de_trabajo_code         25287 non-null int64
nombre_zona_code             25287 non-null int64
dtypes: int64(6)
memory usage: 1.2 MB


In [6]:
testingSet = pd.merge(testingSet,avisos_detalle,how='left',on='idaviso')

In [7]:
testingSet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 8 columns):
id                           100000 non-null int64
idaviso                      100000 non-null int64
idpostulante                 100000 non-null object
nombre_area_code             100000 non-null int64
denominacion_empresa_code    100000 non-null int64
nivel_laboral_code           100000 non-null int64
tipo_de_trabajo_code         100000 non-null int64
nombre_zona_code             100000 non-null int64
dtypes: int64(7), object(1)
memory usage: 6.9+ MB


# Agrego informacion de postulantes

In [8]:
dtypes = pd.Series({'estado': CategoricalDtype(categories=['Abandonado', 'En Curso', 'Graduado'], ordered=False), 'idpostulante': np.dtype('object'),
                    'nombre': CategoricalDtype(categories=['Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario',
                  'Terciario/Técnico', 'Universitario'],ordered=False),'idpostulante': np.dtype('object'),
                    'sexo': CategoricalDtype(categories=['FEM', 'MASC', 'NO_DECLARA'], ordered=False)})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/POSTULANTES_FINAL.csv',dtype=column_types)
postulantes.drop('Unnamed: 0',inplace=True,axis=1)

In [9]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438357 entries, 0 to 438356
Data columns (total 5 columns):
idpostulante    438357 non-null object
edad            438357 non-null int64
estado_code     438357 non-null int64
sexo_code       438357 non-null int64
nombre_code     438357 non-null int64
dtypes: int64(4), object(1)
memory usage: 16.7+ MB


In [10]:
testingSet = pd.merge(testingSet,postulantes,how='left',on='idpostulante')

In [11]:
testingSet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
id                           100000 non-null int64
idaviso                      100000 non-null int64
idpostulante                 100000 non-null object
nombre_area_code             100000 non-null int64
denominacion_empresa_code    100000 non-null int64
nivel_laboral_code           100000 non-null int64
tipo_de_trabajo_code         100000 non-null int64
nombre_zona_code             100000 non-null int64
edad                         93346 non-null float64
estado_code                  93346 non-null float64
sexo_code                    93346 non-null float64
nombre_code                  93346 non-null float64
dtypes: float64(4), int64(7), object(1)
memory usage: 9.9+ MB


In [12]:
testingSet_2 = testingSet[['nombre_area_code',
       'denominacion_empresa_code', 'nivel_laboral_code',
       'tipo_de_trabajo_code', 'nombre_zona_code', 'edad', 'estado_code',
       'sexo_code', 'nombre_code']]

# Utilizamos imputing para los valores que no tenemos

In [13]:
fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=1)
imputed_DF_mean = pd.DataFrame(fill_NaN.fit_transform(testingSet_2))
imputed_DF_mean.columns = ['nombre_area_code',
       'denominacion_empresa_code', 'nivel_laboral_code',
       'tipo_de_trabajo_code', 'nombre_zona_code', 'edad', 'estado_code','sexo_code', 'nombre_code']
imputed_DF_mean.index = testingSet.index

In [14]:
imputed_DF_mean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 9 columns):
nombre_area_code             100000 non-null float64
denominacion_empresa_code    100000 non-null float64
nivel_laboral_code           100000 non-null float64
tipo_de_trabajo_code         100000 non-null float64
nombre_zona_code             100000 non-null float64
edad                         100000 non-null float64
estado_code                  100000 non-null float64
sexo_code                    100000 non-null float64
nombre_code                  100000 non-null float64
dtypes: float64(9)
memory usage: 7.6 MB


In [15]:
fill_NaN = Imputer(missing_values=np.nan, strategy='median', axis=1)
imputed_DF_median = pd.DataFrame(fill_NaN.fit_transform(testingSet_2))
imputed_DF_median.columns = ['nombre_area_code',
       'denominacion_empresa_code', 'nivel_laboral_code',
       'tipo_de_trabajo_code', 'nombre_zona_code', 'edad', 'estado_code','sexo_code', 'nombre_code']
imputed_DF_median.index = testingSet.index

In [16]:
imputed_DF_median.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 9 columns):
nombre_area_code             100000 non-null float64
denominacion_empresa_code    100000 non-null float64
nivel_laboral_code           100000 non-null float64
tipo_de_trabajo_code         100000 non-null float64
nombre_zona_code             100000 non-null float64
edad                         100000 non-null float64
estado_code                  100000 non-null float64
sexo_code                    100000 non-null float64
nombre_code                  100000 non-null float64
dtypes: float64(9)
memory usage: 7.6 MB


In [17]:
fill_NaN = Imputer(missing_values=np.nan, strategy='most_frequent', axis=1)
imputed_DF_most_frequent = pd.DataFrame(fill_NaN.fit_transform(testingSet_2))
imputed_DF_most_frequent.columns = ['nombre_area_code',
       'denominacion_empresa_code', 'nivel_laboral_code',
       'tipo_de_trabajo_code', 'nombre_zona_code', 'edad', 'estado_code','sexo_code', 'nombre_code']
imputed_DF_most_frequent.index = testingSet.index

In [18]:
imputed_DF_most_frequent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 9 columns):
nombre_area_code             100000 non-null float64
denominacion_empresa_code    100000 non-null float64
nivel_laboral_code           100000 non-null float64
tipo_de_trabajo_code         100000 non-null float64
nombre_zona_code             100000 non-null float64
edad                         100000 non-null float64
estado_code                  100000 non-null float64
sexo_code                    100000 non-null float64
nombre_code                  100000 non-null float64
dtypes: float64(9)
memory usage: 7.6 MB


# Agrego las columnas id idaviso e idpostulante

In [20]:
imputed_DF_mean[['id','idaviso','idpostulante']] = testingSet[['id','idaviso','idpostulante']]

In [21]:
imputed_DF_mean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
nombre_area_code             100000 non-null float64
denominacion_empresa_code    100000 non-null float64
nivel_laboral_code           100000 non-null float64
tipo_de_trabajo_code         100000 non-null float64
nombre_zona_code             100000 non-null float64
edad                         100000 non-null float64
estado_code                  100000 non-null float64
sexo_code                    100000 non-null float64
nombre_code                  100000 non-null float64
id                           100000 non-null int64
idaviso                      100000 non-null int64
idpostulante                 100000 non-null object
dtypes: float64(9), int64(2), object(1)
memory usage: 9.9+ MB


In [22]:
imputed_DF_median[['id','idaviso','idpostulante']] = testingSet[['id','idaviso','idpostulante']]
imputed_DF_median.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
nombre_area_code             100000 non-null float64
denominacion_empresa_code    100000 non-null float64
nivel_laboral_code           100000 non-null float64
tipo_de_trabajo_code         100000 non-null float64
nombre_zona_code             100000 non-null float64
edad                         100000 non-null float64
estado_code                  100000 non-null float64
sexo_code                    100000 non-null float64
nombre_code                  100000 non-null float64
id                           100000 non-null int64
idaviso                      100000 non-null int64
idpostulante                 100000 non-null object
dtypes: float64(9), int64(2), object(1)
memory usage: 9.9+ MB


In [23]:
imputed_DF_most_frequent[['id','idaviso','idpostulante']] = testingSet[['id','idaviso','idpostulante']]
imputed_DF_most_frequent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
nombre_area_code             100000 non-null float64
denominacion_empresa_code    100000 non-null float64
nivel_laboral_code           100000 non-null float64
tipo_de_trabajo_code         100000 non-null float64
nombre_zona_code             100000 non-null float64
edad                         100000 non-null float64
estado_code                  100000 non-null float64
sexo_code                    100000 non-null float64
nombre_code                  100000 non-null float64
id                           100000 non-null int64
idaviso                      100000 non-null int64
idpostulante                 100000 non-null object
dtypes: float64(9), int64(2), object(1)
memory usage: 9.9+ MB


# A csvs

In [25]:
imputed_DF_mean.to_csv('TestingSets/testingSet_imp_mean.csv')

In [26]:
imputed_DF_median.to_csv('TestingSets/testingSet_imp_median.csv')

In [27]:
imputed_DF_most_frequent.to_csv('TestingSets/testingSet_imp_most_frequent.csv')